In [17]:
import pandas as pd
import os
import getpass
from pinecone import Pinecone
import cohere

In [18]:
df = pd.read_csv('updated_programs (1).csv')

In [ ]:
COHERE_API_KEY=""
PINECONE_API_KEY=""

In [20]:
co = cohere.Client(os.environ["COHERE_API_KEY"])


In [23]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"], environment="gcp-starter")  # change env if needed


In [26]:
index_name = "programs"
index = pc.Index(index_name)


c:\Users\Madan\Desktop\New folder\Akhila\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
texts = []
metadata_list = []

for _, row in df.iterrows():
    combined_text = f"{row['title']}. {row['description']} Skills: {row['skills_required']}"
    texts.append(combined_text)
    metadata_list.append({
        "program_id": str(row["program_id"]),
        "title": row["title"],
        "category": row["category"],
        "start_date": row["start_date"],
        "end_date": row["end_date"],
        "cost": float(row["cost"]),
        "skills_required": row["skills_required"]
    })


In [31]:
print("Generating embeddings...")
embeds = co.embed(
    texts=texts,
    model="embed-english-v3.0",
    input_type="search_document"  # because you're embedding program data
).embeddings


Generating embeddings...


In [32]:
to_upsert = []
for i, vector in enumerate(embeds):
    vector_id = f"program-{metadata_list[i]['program_id']}"
    to_upsert.append((vector_id, vector, metadata_list[i]))


In [33]:
batch_size = 50
for i in range(0, len(to_upsert), batch_size):
    index.upsert(vectors=to_upsert[i:i+batch_size])


In [34]:
print(f"✅ Successfully uploaded {len(to_upsert)} program vectors to Pinecone.")


✅ Successfully uploaded 50 program vectors to Pinecone.
